## Importa bibliotecas

In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from pprint import pprint
import matplotlib.pyplot as plt
from scipy import stats

##  Cria conexões com os bancos de dado

## Baixando banco de dados

### Corrigindo banco de dados bdworgs


In [2]:
bdworgs = pd.read_excel("./bdworgs.xlsx") # Banco de dados com correções é importado
bdworgs.Num_UG = bdworgs.Num_UG.astype(str) 
bdworgs.to_feather("./bdworgs_corrigido.feather") 

In [3]:
def corrige_ug(ug): # Função criada para corrigir nomes das UGs
    ug = ug.replace(" ","")
    ug = ug.replace("e",",")
    ug = ug.replace("-","a")
    ug = re.sub(r",+",r",",ug)
    ug = re.sub(r"([,ae])(0)",r"\1",ug)
    ug = re.sub(r"^0",r"",ug)
    ug = ug.split("a")
    lista = []
    for i in range(len(ug)-1):
        menor = int(ug[i].split(",")[-1])
        maior = int(ug[i+1].split(",")[0])
        list_nums = list(range(menor+1,maior))
        lista += list_nums
    ug = [el.split(",") for el in ug]
    ug = [int(el2) for el1 in ug for el2 in el1]
    ug+= lista
    return (sorted(ug))

### Carrega bancos de dados

In [4]:
# Carrega bdworgs
bdworgs = pd.read_feather("./bdworgs_corrigido.feather")

# Carrega três bancos de dados do skate
skate_ug = pd.read_feather("./skate_ug.feather")
skate_usinas = pd.read_feather("./skate_usinas.feather")
skate_leilao = pd.read_feather("skate_leilao.feather")

In [5]:
skate_leilao.sort_values(by="DatInicioSuprimento",inplace=True)
skate_leilao.drop_duplicates(keep="first",subset= ["IdeUsinaOutorga"],inplace=True)

skate_usinas = pd.merge(skate_usinas,
        skate_leilao[["DatInicioSuprimento","IdeUsinaOutorga"]],
        on="IdeUsinaOutorga")


mask = skate_usinas.DscComercializacaoEnergia == "ACR"
skate_usinas[["DscComercializacaoEnergia","DatInicioSuprimento"]].sample(10)

,DscComercializacaoEnergia,DatInicioSuprimento
2274,Fora do ACR,None
1808,Fora do ACR,None
506,Fora do ACR,None
362,ACR,2023-01-01
1662,Fora do ACR,None
2203,Fora do ACR,None
1767,Fora do ACR,None
2358,Fora do ACR,None
1074,Fora do ACR,None
253,ACR,2024-01-01


### No dataframe bdworgs são feitas as seguintes alterações:
 - Onde não não existe data na coluna DatMontagemOutorgado e existe na coluna DatConclusaoTorresOutorgado, o valor que está na coluna DatConclusaoTorresOutorgado passa para a coluna DatMontagemOutorgado.
 - Mesmo processo feito para as colunas DatConclusaoTorresOutorgado/DatMontagemOutorgado é feito para as colunas DatConclusaoTorresRealizado/DatMontagemRealizado

In [6]:
mask = bdworgs.Montagem_Outorga.isnull() 
bdworgs.loc[mask,"Montagem_Outorga"] = bdworgs.loc[mask,"Torres_Outorga"]

mask = bdworgs.Montagem_Outorga.isnull() 
bdworgs.loc[mask,"Montagem_Real"] = bdworgs.loc[mask,"Torres_Real"]

### Tabela com associações de nomes entre os bancos de dados

In [7]:
# Carrega tabela de associações
associassoes = pd.read_excel("associassoes.xlsx")
associassoes

,bdworgs,skate
0,Idempreendimento,IdeUsinaOutorga
1,OpTest_Outor_UG,DatInicioOpTesteOutorgada
2,OpCom_Outor_UG,DatUGInicioOpComerOutorgado
3,OpTest_R_UG,DatLiberacaoSFGTeste
4,OpCom_R_UG,DatLiberOpComerRealizado
5,NaN,DatOutorgaUsina
6,UF,UFUsina
7,NaN,SigTipoGeracao
8,PotenciaOutorgada,VlrPotenciaOutorgada
9,Obras_Outorga,DatInicioObraOutorgado


In [8]:
# Cria dicionário dos nomes das colunas para poder relacionar os nomes das colunas
# Dicionário de colunas bdworgs > skate
dict_bdworgs_skate = dict(zip(associassoes.dropna()["bdworgs"],associassoes.dropna()["skate"]))
# Dicionário de colunas skate > bdworgs
dict_skate_bdworgs = dict(zip(associassoes.dropna()["skate"],associassoes.dropna()["bdworgs"]))

In [9]:
# Colunas carregados do skate
skate_cols = [el for el in associassoes["skate"].dropna()]
# Colunas carregadas do bdworgs
bdworgs_cols =[el for el in associassoes["bdworgs"].dropna()]
skate_ug_cols = []
skate_usinas_col = []
for col in skate_cols:
    if col in skate_ug.columns:
        skate_ug_cols.append(col)
    if col in skate_usinas.columns:
        skate_usinas_col.append(col)
        
bdworgs = bdworgs[bdworgs_cols]
skate_ug = skate_ug[skate_ug_cols]
skate_usinas = skate_usinas[skate_usinas_col]
bdworgs = bdworgs.rename(columns=dict_bdworgs_skate)
bdworgs.sample(5)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,DatMontagemRealizado,Torres_Outorga,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,DscComercializacaoEnergia,Num_UG,Pot_UG
1982,29588,2008-06-03,2008-07-15,2008-05-12,2008-07-22,GO,80.0,None,2007-05-07,None,...,2007-11-24,None,None,None,None,None,None,ACR,1,40.0
2296,30285,2012-04-10,None,2011-09-16,2011-10-06,RN,26.0,2011-03-21,2011-03-25,2012-02-01,...,2011-08-09,None,2011-07-23,None,None,None,None,ACR,2,14.0
145,30556,2014-11-01,2014-11-15,2017-02-04,2017-06-15,MG,4.0,2012-03-30,2015-02-28,2012-09-30,...,2015-06-30,None,None,2014-10-31,None,None,2014-06-20,Fora do ACR,01,2.0
192,30642,2014-05-01,2014-06-01,2016-05-14,2016-06-30,MT,21.8,None,2012-04-15,2012-10-01,...,2013-10-01,None,None,2014-02-01,2014-03-02,2013-01-01,2014-03-01,ACR,02,10.3
1972,29555,2008-12-22,2009-01-01,2010-02-27,2010-04-20,BA,150.0,2008-01-07,2008-01-15,None,...,None,None,None,None,None,None,None,ACR,3 e 4,80.0


In [10]:
skate_usinas_col

['IdeUsinaOutorga',
 'DatOutorgaUsina',
 'UFUsina',
 'SigTipoGeracao',
 'VlrPotenciaOutorgada',
 'DatInicioObraOutorgado',
 'DatInicioObraRealizado',
 'DatConcretagemOutorgado',
 'DatConcretagemRealizado',
 'DatMontagemOutorgado',
 'DatMontagemRealizado',
 'DatEnchimentoOutorgado',
 'DatEnchimentoRealizado',
 'DatSisTransmissaoOutorgado',
 'DatSisTransmissaoRealizado',
 'DatInicioSuprimento',
 'DscComercializacaoEnergia']

### Adiciona as seguintes  colunas no bdworgs:
- Quant_UG  &rarr; Quantidade de UGs agrupadas
- SigTipoGeracao &rarr; Tipo de geração


In [11]:
# Insere coluna Quant_UG em bdworgs
bdworgs.insert(3,"Quant_UG",bdworgs["Num_UG"].apply(lambda x : len(corrige_ug(x))))
# Remove coluna Num_UG
bdworgs = bdworgs.drop("Num_UG",axis=1)

ceg_df = pd.read_excel("./Relatorio_Outorgas.xlsx")[["Identificador da Usina","Tipo de Geração"]]
dict_id_ceg = dict(zip(ceg_df["Identificador da Usina"],ceg_df["Tipo de Geração"]))
bdworgs["SigTipoGeracao"] = bdworgs["IdeUsinaOutorga"].map(dict_id_ceg)
bdworgs.sample(5)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,Quant_UG,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,...,DatMontagemRealizado,Torres_Outorga,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,DscComercializacaoEnergia,Pot_UG,SigTipoGeracao
1273,34743,2019-10-16,2020-02-20,1,2019-06-19,2019-07-27,SC,3.300,2018-03-15,2017-11-15,...,2018-12-15,None,None,2019-09-01,2019-01-25,2019-01-15,2018-08-15,ACR,1.665,PCH
598,31438,2017-11-21,2018-01-01,1,2016-05-11,2016-07-12,MS,8.838,2016-04-01,2014-09-01,...,2015-08-30,None,None,2017-10-10,2016-03-23,2016-06-01,2015-06-30,Fora do ACR,4.419,PCH
331,30860,2014-06-01,2014-07-01,14,2016-01-20,2016-02-20,RN,28.000,2013-11-01,2014-06-30,...,2014-11-20,2014-03-01,2014-10-01,None,None,2013-12-01,2014-09-29,ACR,28.000,EOL
659,31601,2015-09-06,2016-01-01,1,2016-01-01,2016-03-01,RS,14.320,2014-09-30,2015-03-28,...,None,2015-06-18,2015-11-02,None,None,2014-10-22,2014-11-15,ACR,1.790,EOL
359,30898,2015-12-01,2015-12-31,1,2014-10-22,2015-03-24,RN,30.000,2015-05-01,2014-01-15,...,2014-10-02,2015-10-01,2014-08-15,None,None,2015-03-01,2013-08-19,ACR,3.000,EOL


### No BD skate_ug, linhas com todas as colunas iguais tais como IdeUsinaOutorga, DatInicioOpTesteOutorgada... (exceto  MdaPotenciaUnitaria, que não necessariamente tem que ser igual em todas as linhas)  são agrupadas e são criadas duas outras colunas: 
- Quant_UG -> &rarr; Quantidade de UGs agrupadas
- Pot_UG -> &rarr; Soma da potência das UGs que foram agrupadas

#### Entrada: skate_ug
#### Saída: skate_ug_grouped

In [12]:
skate_ug_group_list = [x for x  in skate_ug.columns.to_list() if x not in ["MdaPotenciaUnitaria"]]
skate_ug_grouped = skate_ug.groupby(skate_ug_group_list,dropna=False).agg(
    Quant_UG = ("MdaPotenciaUnitaria","count"),
    Pot_UG = ("MdaPotenciaUnitaria","sum")
    
).reset_index()

### Realiza o processo acima para o bdworgs:
- Quant_UG -> &rarr; Quantidade de UGs agrupadas. Nesse caso, como o bdworgs, já possuía a coluna Quant_UG, os valores dessa coluna foram somados para os valores agrupados.
- Pot_UG -> &rarr; Soma da potência dos conjuntos de UGs que foram agrupados.

#### Entrada: bdworgs
#### Saída: bdworgs_ug_grouped

In [13]:
dummy_bdworgs = bdworgs.rename(columns={"Quant_UG":"Quant_UG_","Pot_UG":"Pot_UG_"})
dummy_bdworgs_group_list = [x for x  in dummy_bdworgs.columns.to_list() if x not in ["Quant_UG_","Pot_UG_"]]

bdworgs_ug_grouped = dummy_bdworgs.groupby(dummy_bdworgs_group_list,dropna=False).agg(
    Quant_UG = ("Quant_UG_","sum"),
    Pot_UG = ("Pot_UG_","sum")
    
).reset_index() 

### Checa se a quantidade de potência e de usinas nos dataframes iniciais e agrupados são iguais.

In [14]:
print(skate_ug_grouped.Quant_UG.sum() - skate_ug.MdaPotenciaUnitaria.count())
print(skate_ug_grouped.Pot_UG.sum() - skate_ug.MdaPotenciaUnitaria.sum())
print(bdworgs_ug_grouped.Pot_UG.sum() - bdworgs_ug_grouped.Pot_UG.sum())
print(bdworgs_ug_grouped.Quant_UG.count() - bdworgs_ug_grouped.Quant_UG.count())


0
-1.4901161193847656e-08
0.0
0


### Junta bancos de dados skate_ug e skate_usina pela coluna IdeUsinaOutorga. Todos os atributos de cada usina são passados para cada UG pertecente a usina.

#### Entrada: skate_ug_grouped & skate_usinas
#### Saída: skate_merged

In [15]:
skate_merged  = pd.merge(skate_ug_grouped.drop(["SigTipoGeracao"],axis=1),
        skate_usinas,
        on="IdeUsinaOutorga",
         how="outer")
mask  = skate_merged.DscComercializacaoEnergia == "ACR"
skate_merged.loc[mask,"DatUGInicioOpComerOutorgado"] = skate_merged.DatInicioSuprimento
skate_merged.sample(10)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,Quant_UG,Pot_UG,DatOutorgaUsina,UFUsina,SigTipoGeracao,...,DatConcretagemOutorgado,DatConcretagemRealizado,DatMontagemOutorgado,DatMontagemRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,DatInicioSuprimento,DscComercializacaoEnergia
1061,34781,2023-01-27,2023-02-13,NaN,NaN,8,60000.00,2020-05-26,BA,EOL,...,2022-04-13,None,2022-07-14,None,None,None,2022-02-13,None,None,Fora do ACR
2989,48516,2022-09-23,2022-10-28,NaN,NaN,8,35200.00,2020-11-10,PI,EOL,...,2022-03-11,2022-02-15,2022-08-22,None,None,None,2022-01-19,2021-07-15,None,Fora do ACR
927,34509,2022-12-02,2023-01-01,2021-07-23,2021-11-13,3,12600.00,2018-05-16,BA,EOL,...,2022-05-01,2020-11-29,2022-08-01,2021-05-14,None,None,2022-04-01,2021-01-15,2023-01-01,ACR
3272,49649,2023-12-01,2024-01-01,NaN,NaN,5,5150.00,2021-08-24,SP,UFV,...,None,None,2023-02-01,None,None,None,2023-03-01,None,None,Fora do ACR
2867,47342,2023-05-15,2023-06-15,NaN,NaN,12,41244.00,2021-06-15,MG,UFV,...,None,None,2022-10-15,None,None,None,2022-10-15,None,None,Fora do ACR
2731,47160,2023-09-30,2023-10-31,NaN,NaN,6,25200.00,2022-01-18,RN,EOL,...,2022-07-27,None,2023-01-25,None,None,None,2022-07-31,None,None,Fora do ACR
1364,35834,2019-04-25,2017-01-01,2020-02-07,2021-02-20,14,4928.00,2016-08-16,AM,UTE,...,None,None,2019-02-01,2019-08-22,None,None,2019-02-13,2019-07-15,2017-01-01,ACR
3032,48754,2025-07-01,2025-07-31,NaN,NaN,14,45000.06,2022-04-26,PI,UFV,...,None,None,2024-11-29,None,None,None,2024-11-29,None,None,Fora do ACR
770,33836,2023-08-30,2023-10-15,NaN,NaN,1,6200.00,2021-05-25,RN,EOL,...,2022-10-05,2022-01-18,2023-07-30,None,None,None,2021-12-31,2021-12-01,None,Fora do ACR
1880,38112,2022-10-24,2022-11-29,NaN,NaN,9,49500.00,2020-07-07,PI,EOL,...,2022-02-08,None,2022-08-01,None,None,None,2021-08-17,2021-06-15,None,Fora do ACR


### Converte colunas cujos nomes iniciam em "Dat" para tipo de data. É necessário que as colunas estejam nomeadas no padrão do skate.

In [16]:
bdworgs.DscComercializacaoEnergia.unique()

array(['ACR', 'Fora do ACR'], dtype=object)

In [17]:
skate_usinas.DscComercializacaoEnergia.unique()

array(['ACR', 'Fora do ACR'], dtype=object)

In [18]:
colunas_datas = []
for col in bdworgs_ug_grouped.columns:
    if col[0:3] == "Dat":
        colunas_datas.append(col)
        print(col)
        skate_merged[col] = pd.to_datetime(skate_merged[col],format="%Y-%m-%d",errors="coerce")
        bdworgs_ug_grouped[col] = pd.to_datetime(bdworgs_ug_grouped[col],format="%Y-%m-%d",errors="coerce")

DatInicioOpTesteOutorgada
DatUGInicioOpComerOutorgado
DatLiberacaoSFGTeste
DatLiberOpComerRealizado
DatInicioObraOutorgado
DatInicioObraRealizado
DatConcretagemOutorgado
DatConcretagemRealizado
DatMontagemOutorgado
DatMontagemRealizado
DatEnchimentoOutorgado
DatEnchimentoRealizado
DatSisTransmissaoOutorgado
DatSisTransmissaoRealizado


### Checa quais colunas não estão presentes em cada um dos dataframes

In [19]:
print("Colunas que estão no skate, porém não no bdworgs:")
for col in skate_merged.columns:
    if col not in bdworgs.columns:
        print(col)
        
print("-"*10)
print("Colunas que estão no bdworgs, porém não no skate:")
for col in bdworgs.columns:
    if col not in skate_merged.columns:
        print(col)

Colunas que estão no skate, porém não no bdworgs:
DatOutorgaUsina
DatInicioSuprimento
----------
Colunas que estão no bdworgs, porém não no skate:
Torres_Outorga
Torres_Real


In [20]:
skate_merged

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,Quant_UG,Pot_UG,DatOutorgaUsina,UFUsina,SigTipoGeracao,...,DatConcretagemOutorgado,DatConcretagemRealizado,DatMontagemOutorgado,DatMontagemRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,DatInicioSuprimento,DscComercializacaoEnergia
0,273,2020-12-06,2024-01-01,2002-06-15,2002-06-15,1,4000.0,2018-08-13,PR,PCH,...,2022-03-15,2021-10-14,2022-08-08,2022-02-15,NaT,NaT,2022-02-12,2022-02-15,2024-01-01,ACR
1,273,2021-02-02,2024-01-01,NaT,NaT,1,8000.0,2018-08-13,PR,PCH,...,2022-03-15,2021-10-14,2022-08-08,2022-02-15,NaT,NaT,2022-02-12,2022-02-15,2024-01-01,ACR
2,273,2023-08-03,2024-01-01,2002-09-15,2002-09-15,1,4000.0,2018-08-13,PR,PCH,...,2022-03-15,2021-10-14,2022-08-08,2022-02-15,NaT,NaT,2022-02-12,2022-02-15,2024-01-01,ACR
3,273,2023-09-02,2024-01-01,NaT,NaT,1,8000.0,2018-08-13,PR,PCH,...,2022-03-15,2021-10-14,2022-08-08,2022-02-15,NaT,NaT,2022-02-12,2022-02-15,2024-01-01,ACR
4,324,2015-05-31,2015-06-30,NaT,NaT,1,7500.0,2003-08-12,RS,UHE,...,2014-12-31,NaT,2015-03-31,NaT,NaT,NaT,2014-10-31,NaT,None,Fora do ACR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3718,56350,2022-04-27,2022-05-01,NaT,NaT,1,9370.0,2021-11-05,ES,UTE,...,NaT,NaT,2022-04-25,2022-04-01,NaT,NaT,2022-02-25,2021-10-25,2022-05-01,ACR
3719,56350,2022-04-28,2022-05-01,NaT,NaT,1,9370.0,2021-11-05,ES,UTE,...,NaT,NaT,2022-04-25,2022-04-01,NaT,NaT,2022-02-25,2021-10-25,2022-05-01,ACR
3720,56350,2022-04-29,2022-05-01,NaT,NaT,1,9370.0,2021-11-05,ES,UTE,...,NaT,NaT,2022-04-25,2022-04-01,NaT,NaT,2022-02-25,2021-10-25,2022-05-01,ACR
3721,56665,2023-03-30,2023-05-30,NaT,NaT,1,9360.0,2022-04-12,BA,UTE,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,None,Fora do ACR


### Junta BDs skate e bdworgs.
#### Entrada: skate_merged & bdworgs_ug_grouped
#### Saída: skate_merged

In [21]:
bdworgs_ug_grouped["origem"] = "bdworgs"
skate_merged["origem"] = "skate"
bd_combined = pd.concat([bdworgs_ug_grouped,skate_merged],axis=0,ignore_index=True)
#bd_combined = bdworgs_ug_grouped.copy(deep=True)
bd_combined["origem"] = bd_combined["origem"].astype("category")
bd_combined.DscComercializacaoEnergia = bd_combined.DscComercializacaoEnergia.astype("category")
bd_combined.sample(10)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,DscComercializacaoEnergia,SigTipoGeracao,Quant_UG,Pot_UG,origem,DatOutorgaUsina,DatInicioSuprimento
3873,37381,2024-10-01,2025-01-01,NaT,NaT,ES,8000.00,2023-06-01,NaT,2023-12-01,...,NaT,2024-06-01,NaT,ACR,PCH,1,4000.00,skate,2020-04-06,2025-01-01
1196,30750,2013-06-23,2013-06-30,2015-08-05,2015-08-27,RS,25.20,NaT,2012-08-01,2012-04-07,...,NaT,2012-07-06,2012-03-01,ACR,EOL,12,25.20,bdworgs,NaN,NaN
1346,30959,2013-09-03,2013-10-01,2013-12-14,2014-04-04,MA,254.84,NaT,2012-10-30,NaT,...,NaT,NaT,NaT,Fora do ACR,UTE,1,127.42,bdworgs,NaN,NaN
1670,31666,2017-11-01,2017-12-01,2016-09-28,2016-10-11,PI,22.00,2016-04-01,2015-03-02,2016-07-01,...,NaT,2016-07-01,2015-03-18,ACR,EOL,2,4.40,bdworgs,NaN,NaN
1242,30800,2013-11-25,2014-02-01,2014-02-22,2016-07-01,MA,518.80,2012-09-23,2012-09-15,NaT,...,NaT,2013-04-30,2013-04-15,ACR,UTE,1,168.80,bdworgs,NaN,NaN
1092,30546,2014-09-10,2014-10-10,2014-09-10,2014-10-11,BA,30.24,2011-11-30,2013-06-15,2011-12-15,...,NaT,2012-01-30,2013-05-01,ACR,EOL,18,30.24,bdworgs,NaN,NaN
5011,46791,2023-10-01,2023-11-01,NaT,NaT,MG,13748.00,2022-11-01,NaT,NaT,...,NaT,2023-02-01,NaT,Fora do ACR,UFV,4,13748.00,skate,2021-08-24,None
5028,46889,2023-04-20,2023-05-30,NaT,NaT,BA,28000.00,2022-08-01,NaT,NaT,...,NaT,2022-12-15,NaT,Fora do ACR,UFV,14,28000.00,skate,2021-10-05,None
3725,36974,2022-11-15,2023-01-01,2020-01-21,2020-02-29,RN,31185.00,2022-03-18,2019-09-18,2022-05-18,...,NaT,2022-04-05,2018-11-03,ACR,EOL,2,6930.00,skate,2018-05-30,2023-01-01
4984,46721,2022-11-01,2023-01-01,NaT,NaT,BA,32740.00,2021-06-01,NaT,NaT,...,NaT,2022-06-01,NaT,Fora do ACR,UFV,21,32739.82,skate,2021-03-23,None


In [22]:
bd_combined.to_parquet("bd_combined.gzip")